# Challenge Exercises

1. Improve timer decorator to log fastest and slowest durations
2. Create a debug decorator that enters debugger before function call
3. Implement a retry decorator to retry failing function calls
4. Enhance cache with maximum size and LRU cache expiration
5. Add parameterization so delays can be customized

In [5]:
# improve timer decorator to log fastest and slowest functions
# hint: use timeit

import functools
import time

# Dictionary to store execution times
execution_times = {}

def measure_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed_time = time.time() - start
        print(f"{func.__name__} ran in {elapsed_time} secs")
        execution_times[func.__name__] = elapsed_time
        return result
    return wrapper

def get_fastest():
    if not execution_times:
        return None, None
    fastest_func = min(execution_times, key=execution_times.get)
    return fastest_func, execution_times[fastest_func]

def get_slowest():
    if not execution_times:
        return None, None
    slowest_func = max(execution_times, key=execution_times.get)
    return slowest_func, execution_times[slowest_func]

@measure_time
def sum_nums():
    result = 0
    for x in range(1000000):
        result += x

@measure_time
def multiply_nums():
    result = 0
    for x in range(1000000):
        result *= x

@measure_time
def subtract_nums():
    result = 0
    for x in range(1000000, 0, -1):
        result *= x

print("=" * 50)
multiply_nums()
sum_nums()
subtract_nums()
print("=" * 50)

fastest_func, fastest_fun_exec_time = get_fastest()
slowest_func, slowest_func_exec_time = get_slowest()

print(f"Fastest function: {fastest_func} ran in {fastest_fun_exec_time} secs")
print(f"Slowest function: {slowest_func} ran in {slowest_func_exec_time} secs")


multiply_nums ran in 0.026826143264770508 secs
sum_nums ran in 0.028089284896850586 secs
subtract_nums ran in 0.018121957778930664 secs
Fastest function: subtract_nums ran in 0.018121957778930664 secs
Slowest function: sum_nums ran in 0.028089284896850586 secs


In [ ]:
# Create a debug decorator that enters debugger before function call

import functools
import pdb

def debug_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        pdb.set_trace()
        return func(*args, **kwargs)
    return wrapper

@debug_decorator
def sample_function(x, y):
    z = x + y
    print(f"The result is {z}")
    return z

# Call the decorated function
sample_function(3, 5)

In [9]:
# Implement a retry decorator to retry failing function calls

import time
import functools

def retry_decorator(max_retries=3, delay=1, exceptions=(Exception,)):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_retries:
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    attempts += 1
                    print(f"Attempt {attempts} failed with error: {e}")
                    if attempts < max_retries:
                        time.sleep(delay)
                    else:
                        print("Max retries reached. Raising the exception.")
                        raise
        return wrapper
    return decorator

# Example usage
@retry_decorator(max_retries=5, delay=2, exceptions=(ValueError,))
def unreliable_function():
    if time.time() % 2 > 1:
        raise ValueError("Intermittent error")
    return "Success!"

# Call the decorated function
try:
    result = unreliable_function()
    print(f"Function succeeded with result: {result}")
except ValueError as e:
    print(f"Function failed after retries with error: {e}")

Attempt 1 failed with error: Intermittent error
Attempt 2 failed with error: Intermittent error
Attempt 3 failed with error: Intermittent error
Attempt 4 failed with error: Intermittent error
Attempt 5 failed with error: Intermittent error
Max retries reached. Raising the exception.
Function failed after retries with error: Intermittent error


In [26]:
# Enhance cache with maximum size and LRU cache expiration

# There are two ways to implement this, one is by using python native @functools.lru_cache and the other way is using custom class

import functools
import time

MAX_CACHE_SIZE = 125

@functools.lru_cache(maxsize=MAX_CACHE_SIZE, typed=True)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

def callfibonacci(n):
    start_time = time.perf_counter()
    result = fibonacci(45)
    end_time = time.perf_counter()
    return result, end_time - start_time


# Call the decorated function
print(f"Result of 1st time fibonacci(45) takes more time : {callfibonacci(45)}") # Computed and stored to cache
print(f"Result of 2nd time fibonacci(45) takes less time {callfibonacci(45)}") # Retrieved from cache

print(fibonacci.cache_info())

Result of 1st time fibonacci(45) takes more time : (1134903170, 3.333299537189305e-05)
Result of 2nd time fibonacci(45) takes less time (1134903170, 7.919952622614801e-07)
CacheInfo(hits=44, misses=46, maxsize=125, currsize=46)


In [29]:
# Custom timed LRU cache decorator

import functools
import time

MAX_CACHE_SIZE = 128

def timed_lru_cache(seconds, maxsize=MAX_CACHE_SIZE):
    def decorator(func):
        # Use lru_cache to store results
        func = functools.lru_cache(maxsize=maxsize)(func)
        # Store the insertion time of each cache entry
        func._timestamps = {}
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Current time
            now = time.time()
            # Cache key based on function arguments
            key = functools._make_key(args, kwargs, typed=False)
            
            # Check if the cached value has expired
            if key in func._timestamps and now - func._timestamps[key] > seconds:
                # Invalidate the cache entry
                func.cache_clear()
                func._timestamps.pop(key, None)
                
            # Call the function (either from cache or recompute)
            result = func(*args, **kwargs)
            
            # Update the timestamp
            func._timestamps[key] = now
            return result

        # Add a method to clear all timestamps (if needed)
        def cache_clear():
            func.cache_clear()
            func._timestamps.clear()
        
        wrapper.cache_clear = cache_clear
        wrapper.cache_info = func.cache_info
        return wrapper
    return decorator

# Example usage
@timed_lru_cache(seconds=5, maxsize=128)
def slow_function(x):
    time.sleep(2)
    return x * 2

@timed_lru_cache(seconds=5, maxsize=MAX_CACHE_SIZE)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

def callfibonacci(n):
    start_time = time.perf_counter()
    result = fibonacci(45)
    end_time = time.perf_counter()
    return result, end_time - start_time

# Call the function and observe the caching behavior
# Call the decorated function
print(f"Result of 1st time fibonacci(45) takes more time : {callfibonacci(45)}") # Computed and cached
print(f"Result of 2nd time fibonacci(45) takes less time {callfibonacci(45)}") # Retrieved from cache

# print(slow_function(2))  # Computed and cached
# print(slow_function(2))  # Retrieved from cache

time.sleep(6)
print(f"Result of 3rd time cache expired fibonacci(45) takes more time {callfibonacci(45)}") # Cache expired, recomputed
# print(slow_function(2))  # Cache expired, recomputed

# Cache information
print(slow_function.cache_info())

Result of 1st time fibonacci(45) takes more time : (1134903170, 0.00012708400026895106)
Result of 2nd time fibonacci(45) takes less time (1134903170, 2.583001332823187e-06)
Result of 3rd time cache expired fibonacci(45) takes more time (1134903170, 0.000244708004174754)
CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)


In [31]:
# Add parameterization so delays can be customized

import time
from functools import wraps

def delay(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"Sleeping for {seconds} seconds before running {func.__name__}")
            time.sleep(seconds)
            return func(*args, **kwargs)
        return wrapper
    return decorator


@delay(seconds=3)
def print_text():
    print("Hello World")

print_text()

Sleeping for 3 seconds before running print_text
Hello World
